In [1]:
import pandas as pd
import torch.nn as nn
import torch
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.nn import BatchNorm1d
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from tqdm.auto import tqdm

In [2]:
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-12):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias

In [10]:

class SelfAttention(nn.Module):
    def __init__(self, num_attention_heads, input_size, hidden_size, output_dim=1, kernel_size=3,
                 hidden_dropout_prob=0.5, attention_probs_dropout_prob=0.5):
        super(SelfAttention, self).__init__()
        input_size = input_size - kernel_size + 1

        if hidden_size % num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (hidden_size, num_attention_heads))

        self.num_attention_heads = num_attention_heads
        self.attention_head_size = int(hidden_size / num_attention_heads)
        self.all_head_size = hidden_size

        self.query = nn.Linear(input_size, self.all_head_size)
        self.key = nn.Linear(input_size, self.all_head_size)
        self.value = nn.Linear(input_size, self.all_head_size)

        self.attn_dropout = nn.Dropout(attention_probs_dropout_prob)
        self.out_dropout = nn.Dropout(hidden_dropout_prob)

        self.dense = nn.Linear(hidden_size, input_size)


        self.LayerNorm = LayerNorm(input_size, eps=1e-12)

        self.out = nn.Linear(input_size, output_dim)

        self.cnn = nn.Conv1d(1, 1, kernel_size, stride=1)
        self.relu = nn.ReLU()
        self.LeakyReLU = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, input_tensor): #input tensor (128,10000)
        cnn_hidden = self.cnn(input_tensor.view(input_tensor.size(0), 1, -1))#(128,1,9998)
        cnn_hidden = self.LeakyReLU(cnn_hidden)

        cnn_hidden = self.dropout(cnn_hidden)

        input_tensor = cnn_hidden
        self.cnn_result = input_tensor

        mixed_query_layer = self.query(input_tensor)
        mixed_key_layer = self.key(input_tensor)
        mixed_value_layer = self.value(input_tensor)

        self.mixed_query_layer_ = mixed_query_layer
        self.mixed_key_layer_ = mixed_key_layer
        self.mixed_value_layer_ = mixed_value_layer

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        self.query_layer_ = query_layer
        self.key_layer_ = key_layer
        self.value_layer_ = value_layer

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / np.sqrt(self.attention_head_size)
        self.attention_scores_ = attention_scores
        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        self.attention_probs_ = attention_probs
        attention_probs = self.attn_dropout(attention_probs)
        self.attention_probs__ = attention_probs

        context_layer = torch.matmul(attention_probs, value_layer)
        self.context_layer_ = context_layer

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        self.context_layer__ = context_layer

        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        self.new_context_layer_shape_ = new_context_layer_shape

        context_layer = context_layer.view(*new_context_layer_shape)
        self.context_layer3 = context_layer



        hidden_states = self.dense(context_layer)
        self.h1 = hidden_states

        hidden_states = self.out_dropout(hidden_states) #128，1，9998
        self.h2 = hidden_states

        hidden_states = self.LayerNorm(hidden_states + input_tensor) #128，1，9998
        self.h3 = hidden_states

        output = self.out(self.LeakyReLU(hidden_states.view(hidden_states.size(0), -1)))
        self.o = output
        return output


In [12]:
np.random.seed(42)
torch.manual_seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device used: {DEVICE}')
is_scaler = False
batch_size = 128  
hidden_dim = 256  
output_dim = 1  
num_attention_heads = 8
kernel_size = 4
best_corr_coef = -1
LR = 0.0001
epochs = 100
hidden_dropout_prob=0.5
attention_probs_dropout_prob = 0.5

Device used: cuda


In [13]:
data = pd.read_csv("./data_gw.csv")
label = pd.read_csv("./pheno.csv")


In [14]:
data

,12203,13948,30630,35513,26810,42399,47012,5328,15925,13967,...,40530,3979,39572,13851,8183,23558,39900,29722,42787,34352
0,0.782266,-0.823355,0.761606,0.055514,-0.906444,-0.710666,0.230088,0.845777,0.528562,0.957387,...,-0.105683,0.705837,0.478487,-1.235228,0.510296,0.528181,0.098919,0.051824,-0.855337,0.440399
1,0.782266,-0.823355,0.761606,0.055514,-0.906444,-0.710666,0.230088,2.307794,0.528562,0.957387,...,-0.105683,2.382408,0.478487,-0.125908,-0.767040,0.528181,0.098919,0.051824,-0.855337,0.440399
2,0.782266,-0.823355,0.761606,0.055514,-0.906444,-0.710666,0.230088,0.845777,0.528562,0.957387,...,-0.105683,0.705837,0.478487,-1.235228,0.510296,0.528181,0.098919,0.051824,-0.855337,0.440399
3,0.782266,0.525302,-0.940306,0.996435,0.764420,-0.710666,-0.980900,0.845777,-0.849696,0.957387,...,-0.105683,-0.970735,0.478487,0.983412,0.510296,0.528181,0.098919,0.051824,1.169130,1.956404
4,1.674245,0.525302,-0.940306,0.996435,0.764420,-0.710666,-0.980900,0.845777,-0.849696,0.957387,...,-0.105683,-0.970735,-0.945581,-1.235228,0.510296,0.528181,0.098919,0.051824,1.169130,1.956404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.109713,-0.823355,0.761606,-0.885407,-0.906444,0.692426,0.230088,-0.616240,0.528562,-2.247256,...,-0.105683,0.705837,-0.945581,0.983412,-0.767040,0.528181,0.098919,0.051824,-0.855337,-1.075605
1996,1.674245,0.525302,0.761606,0.996435,0.764420,-0.710666,-0.980900,0.845777,-0.849696,-0.644935,...,-0.105683,-0.970735,0.478487,-0.125908,-0.767040,0.528181,0.098919,0.051824,1.169130,0.440399
1997,0.782266,0.525302,2.463518,0.996435,2.435284,-0.710666,2.652063,0.845777,-0.849696,-0.644935,...,-0.105683,0.705837,-0.945581,0.983412,0.510296,0.528181,0.098919,0.051824,1.169130,0.440399
1998,1.674245,0.525302,-0.940306,0.996435,0.764420,-0.710666,-0.980900,0.845777,-0.849696,0.957387,...,-0.105683,0.705837,-0.945581,0.983412,0.510296,-0.724918,0.098919,0.051824,-0.855337,0.440399


In [15]:
label = label["width"]

In [16]:
X_train_reduction, X_test_reduction, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=2)

In [17]:
Y_train = pd.DataFrame(Y_train)
Y_test = pd.DataFrame(Y_test)
Y_train = Y_train.values.astype(float)
Y_test = Y_test.values.astype(float)
X_train_reduction = pd.DataFrame(X_train_reduction)
X_test_reduction = pd.DataFrame(X_test_reduction)
X_train_reduction = X_train_reduction.values.astype(float)
X_test_reduction = X_test_reduction.values.astype(float)

In [18]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_corr_coefs = []

start_time = time.time()
x = X_train_reduction
y = Y_train


In [19]:
save_path = './cn_at_gw.pth'

In [ ]:
start_time = time.time()
x = X_train_reduction
y = Y_train
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]


    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    # scaler_model = StandardScaler()
    # y_train = scaler_model.fit_transform(y_train.reshape(-1, 1)).flatten()
    # y_test = scaler_model.transform(y_test.reshape(-1, 1)).flatten()

    x_train_tensor = torch.from_numpy(x_train).float().to(DEVICE)
    y_train_tensor = torch.from_numpy(y_train).float().to(DEVICE)
    x_test_tensor = torch.from_numpy(x_test).float().to(DEVICE)
    y_test_tensor = torch.from_numpy(y_test).float().to(DEVICE)

    train_data = TensorDataset(x_train_tensor, y_train_tensor)
    test_data = TensorDataset(x_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_data, batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size, shuffle=False)

    model = SelfAttention(num_attention_heads, x_train.shape[1], hidden_dim, output_dim,
                          hidden_dropout_prob=hidden_dropout_prob, kernel_size=kernel_size,
                          attention_probs_dropout_prob=attention_probs_dropout_prob).to(DEVICE)
    loss_function = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=10, verbose=True)

    best_corr_coef = -1

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        train_bar = tqdm(train_loader, desc=f'第 {epoch + 1}/{epochs} 轮')
        for data in train_bar:
            x_batch, y_batch = data
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = loss_function(y_pred, y_batch.reshape(-1, 1))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

    model.eval()
    y_test_preds, y_test_trues = [], []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            y_test_pred = model(x_batch)
            y_test_preds.extend(y_test_pred.cpu().numpy().reshape(-1).tolist())
            y_test_trues.extend(y_batch.cpu().numpy().reshape(-1).tolist())

    corr_coef = np.corrcoef(y_test_preds, y_test_trues)[0, 1]
    if corr_coef > best_corr_coef:
        best_corr_coef = corr_coef
        torch.save(model.state_dict(), save_path)
    best_corr_coefs.append(best_corr_coef)

    print(f'Fold {fold+1} - Epoch {epoch+1}: Best Correlation Coefficient: {best_corr_coef:.4f}')

end_time = time.time()
execution_time = end_time - start_time
print(f"time: {execution_time} 秒")


In [21]:
average = np.mean(best_corr_coefs)
print(best_corr_coefs)
print(f"corr: {average:.4f}")
import numpy as np
performance_scores = np.array(best_corr_coefs)
mean_score = np.mean(performance_scores)
sample_variance = np.var(performance_scores, ddof=1)
sample_std_dev = np.sqrt(sample_variance)
standard_error = sample_std_dev / np.sqrt(len(performance_scores))
print(standard_error)

各折最佳相关系数: [0.6893561082228018, 0.7218105070433697, 0.708406691545386, 0.7119161859195129, 0.6932000359347648]
平均最佳相关系数: 0.7049
0.006024879855239936


In [25]:
### load_model
model_gw = SelfAttention(num_attention_heads, 
                      X_train_reduction.shape[1], 
                      hidden_dim, 
                      output_dim,
                      hidden_dropout_prob=hidden_dropout_prob,
                      kernel_size = kernel_size,
                      attention_probs_dropout_prob=attention_probs_dropout_prob).to(DEVICE)
model_gw.load_state_dict(torch.load('./cn_at_gw.pth'))

<All keys matched successfully>

In [26]:

data_test_tensor = torch.from_numpy(X_test_reduction).to(torch.float32).to(DEVICE)
label_test_tensor = torch.from_numpy(Y_test).to(torch.float32).to(DEVICE)

In [27]:
model_gw.eval()
with torch.no_grad():
    a_output = model_gw(data_test_tensor)
pre = a_output.cpu().detach().numpy().reshape(-1).tolist()
tru = label_test_tensor.cpu().detach().numpy().reshape(-1).tolist()
test_corr_coef = np.corrcoef(pre, tru)[0,1]
print("test_corr",test_corr_coef)

test_corr 0.7103816398643904
